In [0]:
#Load dependenices
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [0]:
#Import file
df = pd.read_csv('https://charlies-angels.s3.us-east-2.amazonaws.com/movie_urls.txt', names = ["url"])

In [0]:
df.nunique()

#Remove duplicates
df = df.drop_duplicates()
df.count()
df.reset_index(inplace = True, drop = True)


#Add a column for Reviews
df['reviews'] = ''

#Add a column for IMDB Movie ID
df['movie_id'] = df['url'].str.replace('/usercomments','').str.replace('http://www.imdb.com/title/','')

In [4]:
df.head(5)

,url,reviews,movie_id
0,http://www.imdb.com/title/tt0018515/usercomments,,tt0018515
1,http://www.imdb.com/title/tt0118866/usercomments,,tt0118866
2,http://www.imdb.com/title/tt0436149/usercomments,,tt0436149
3,http://www.imdb.com/title/tt0050652/usercomments,,tt0050652
4,http://www.imdb.com/title/tt0203895/usercomments,,tt0203895


In [0]:
#Update the URLs to what we want
df['url'] = df['url'].str.replace('usercomments','reviews')

In [6]:
df.head()

,url,reviews,movie_id
0,http://www.imdb.com/title/tt0018515/reviews,,tt0018515
1,http://www.imdb.com/title/tt0118866/reviews,,tt0118866
2,http://www.imdb.com/title/tt0436149/reviews,,tt0436149
3,http://www.imdb.com/title/tt0050652/reviews,,tt0050652
4,http://www.imdb.com/title/tt0203895/reviews,,tt0203895


In [7]:
# Online Movie Database (OMDB) API URL including API Key and full plot
api_url = "http://www.omdbapi.com/?apikey=baee4093&plot=full"

movie_data = []

# Using a for-loop...
for movie_id in df['movie_id']:
    # Use the movie_id to pull data from the API
    try:
        #print(movie_id)
        movie = requests.get(api_url + '&i=' + movie_id).json()
        if movie['Response'] == "True":
            movie_data.append(movie)
    except JSONDecodeError:
        #print(movie_id)
        continue

tt0018515
tt0118866
tt0436149
tt0050652
tt0203895
tt0402644
tt0051885
tt0017735
tt0170996
tt0248123
tt0456526
tt0050916
tt0065755
tt0076650
tt0293944
tt0036811
tt0076029
tt0039277
tt0074532
tt0100263
tt0045992
tt0025254
tt0039041
tt0875630
tt0112443
tt0235712
tt0312296
tt0247729
tt0371226
tt0430283
tt0115685
tt0024327
tt0029952
tt0218542
tt0114367
tt0348914
tt0161644
tt0198871
tt0353489
tt0094029
tt0365393
tt0044811
tt0708842
tt0443584
tt0115006
tt0094375
tt0820466
tt0441758
tt0384573
tt0933342
tt0118747
tt0982935
tt0242527
tt0088392
tt0466766
tt0095342
tt0074329
tt0488338
tt0065922
tt0074871
tt0001013
tt0105984
tt0216670
tt0650630
tt0118515
tt0374180
tt0100369
tt0122933
tt0164143
tt0069301
tt0065424
tt0030833
tt0089087
tt0442781
tt0086247
tt0074053
tt0751107
tt0498366
tt0317310
tt0428856
tt0226765
tt0039500
tt0914797
tt0158423
tt0253457
tt0099316
tt0050454
tt0457600
tt0270291
tt0229480
tt0082677
tt0158049
tt0107423
tt0120122
tt0192638
tt0067345
tt0037386
tt0316946
tt0023761
tt0073233


NameError: ignored

In [0]:
omdb_df = pd.DataFrame(movie_data)

In [0]:
movie_only = omdb_df.copy()
movie_only = movie_only[movie_only['Type'] == 'movie']
movie_only = movie_only.drop_duplicates(subset ="imdbID") 
movie_only = movie_only[movie_only['imdbRating'] != 'N/A']

In [10]:
print(movie_only['Type'].value_counts())
print(movie_only['imdbID'].nunique())

movie    2665
Name: Type, dtype: int64
2665


In [0]:
movie_only['imdbRating'] = pd.to_numeric(movie_only['imdbRating'])

In [12]:
movie_only.head(2)

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,Language,Country,Awards,Poster,Ratings,Metascore,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response,totalSeasons,Season,Episode,seriesID
0,Two Arabian Knights,1927,TV-G,23 Sep 1927,92 min,"Comedy, Romance, Adventure",Lewis Milestone,"Wallace Smith (screen version), Cyril Gardner ...","William Boyd, Mary Astor, Louis Wolheim, Ian K...",Americans Sgt. Peter O'Gaffney and one of his ...,English,USA,N/A,https://m.media-amazon.com/images/M/MV5BNTJiNG...,"[{'Source': 'Internet Movie Database', 'Value'...",N/A,6.9,587,tt0018515,movie,N/A,N/A,Caddo,N/A,True,NaN,NaN,NaN,NaN
1,Clockwatchers,1997,PG-13,15 May 1998,96 min,"Comedy, Drama",Jill Sprecher,"Jill Sprecher, Karen Sprecher","Toni Collette, Parker Posey, Lisa Kudrow, Alan...",Iris can best be described as a wallflower. Sh...,English,"UK, USA",2 wins & 1 nomination.,https://m.media-amazon.com/images/M/MV5BYzZlND...,"[{'Source': 'Internet Movie Database', 'Value'...",N/A,6.5,"4,377",tt0118866,movie,26 Sep 2006,N/A,BMG,N/A,True,NaN,NaN,NaN,NaN


In [13]:
df.head()

,url,reviews,movie_id
0,http://www.imdb.com/title/tt0018515/reviews,,tt0018515
1,http://www.imdb.com/title/tt0118866/reviews,,tt0118866
2,http://www.imdb.com/title/tt0436149/reviews,,tt0436149
3,http://www.imdb.com/title/tt0050652/reviews,,tt0050652
4,http://www.imdb.com/title/tt0203895/reviews,,tt0203895


In [0]:
df = pd.merge(df, movie_only, left_on='movie_id', right_on='imdbID', how='inner')

#Rename imdbRating to label, sinve Naive Bayes requires it
df.rename(columns={'imdbRating' : 'label'}, inplace=True)

In [15]:
df.head()

,url,reviews,movie_id,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,Language,Country,Awards,Poster,Ratings,Metascore,label,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response,totalSeasons,Season,Episode,seriesID
0,http://www.imdb.com/title/tt0018515/reviews,,tt0018515,Two Arabian Knights,1927,TV-G,23 Sep 1927,92 min,"Comedy, Romance, Adventure",Lewis Milestone,"Wallace Smith (screen version), Cyril Gardner ...","William Boyd, Mary Astor, Louis Wolheim, Ian K...",Americans Sgt. Peter O'Gaffney and one of his ...,English,USA,N/A,https://m.media-amazon.com/images/M/MV5BNTJiNG...,"[{'Source': 'Internet Movie Database', 'Value'...",N/A,6.9,587,tt0018515,movie,N/A,N/A,Caddo,N/A,True,NaN,NaN,NaN,NaN
1,http://www.imdb.com/title/tt0118866/reviews,,tt0118866,Clockwatchers,1997,PG-13,15 May 1998,96 min,"Comedy, Drama",Jill Sprecher,"Jill Sprecher, Karen Sprecher","Toni Collette, Parker Posey, Lisa Kudrow, Alan...",Iris can best be described as a wallflower. Sh...,English,"UK, USA",2 wins & 1 nomination.,https://m.media-amazon.com/images/M/MV5BYzZlND...,"[{'Source': 'Internet Movie Database', 'Value'...",N/A,6.5,"4,377",tt0118866,movie,26 Sep 2006,N/A,BMG,N/A,True,NaN,NaN,NaN,NaN
2,http://www.imdb.com/title/tt0436149/reviews,,tt0436149,Buried in the Sand: The Deception of America,2004,N/A,21 Sep 2004,90 min,"Documentary, War",David Wald,Rob Cartee,"Mark Taylor, Ted Kennedy, Saddam Hussein, Geor...",Conservative political commentator Mark Taylor...,English,USA,N/A,https://m.media-amazon.com/images/M/MV5BMjEyND...,"[{'Source': 'Internet Movie Database', 'Value'...",N/A,5.1,56,tt0436149,movie,21 Sep 2004,N/A,N/A,N/A,True,NaN,NaN,NaN,NaN
3,http://www.imdb.com/title/tt0050652/reviews,,tt0050652,The Lonely Man,1957,Approved,21 Jun 1957,88 min,"Drama, Western",Henry Levin,"Harry Essex, Robert Smith","Jack Palance, Anthony Perkins, Neville Brand, ...",Gunslinger Jacob Wade finds his long-abandoned...,English,USA,N/A,https://m.media-amazon.com/images/M/MV5BZDY3MW...,"[{'Source': 'Internet Movie Database', 'Value'...",N/A,6.5,564,tt0050652,movie,22 Apr 2003,N/A,Paramount Pictures,N/A,True,NaN,NaN,NaN,NaN
4,http://www.imdb.com/title/tt0203895/reviews,,tt0203895,Sea Prince and the Fire Child,1981,N/A,18 Jul 1981,108 min,"Animation, Drama, Family, Fantasy, Romance",Masami Hata,"Shintarô Tsuji (story), Masami Hata, Chiho Kat...","Tôru Furuya, Mami Koyama, Keiko Han, Michiko N...",Years after the war between the Children of Wa...,Japanese,Japan,N/A,https://m.media-amazon.com/images/M/MV5BZGJmND...,"[{'Source': 'Internet Movie Database', 'Value'...",N/A,7.2,371,tt0203895,movie,N/A,N/A,N/A,N/A,True,NaN,NaN,NaN,NaN


In [0]:
#Retrieve reviews
for idx in range(len(df)):
    reviews = ''
    
    response = requests.get(df.loc[idx, 'url'])
    soup = BeautifulSoup(response.text, 'lxml')
    results = soup.find_all('div', class_='text show-more__control')
    
    for result in results:
        reviews = reviews + result.text + ' ' 
        #result.find('div', class_='text show-more__control').text.strip()
    
    df.loc[idx, ['reviews']] = reviews
    #break

In [0]:
#Take off
df = df[df['reviews'] != '']

In [0]:
df.head()

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java

!pip install pyspark
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .getOrCreate()

In [0]:
reviews_df = spark.createDataFrame(df)
reviews_df.show()

In [0]:
from pyspark.sql.functions import length
# Create a length column to be used as a future feature 
data_df = reviews_df.withColumn('length', length(reviews_df['reviews']))
data_df.show()

In [0]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer
# Create all the features to the data set
#pos_neg_to_num = StringIndexer(inputCol='class',outputCol='label')
tokenizer = Tokenizer(inputCol="reviews", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
hashingTF = HashingTF(inputCol="stop_tokens", outputCol='hash_token')
idf = IDF(inputCol='hash_token', outputCol='idf_token')

In [0]:

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

# Create feature vectors
#clean_up = VectorAssembler(inputCols=['idf_token', 'length'], outputCol='features')
clean_up = VectorAssembler(inputCols=['idf_token'], outputCol='features')

In [0]:

# Create a and run a data processing Pipeline
from pyspark.ml import Pipeline
data_prep_pipeline = Pipeline(stages=[tokenizer, stopremove, hashingTF, idf, clean_up])

In [0]:
# Fit and transform the pipeline
cleaner = data_prep_pipeline.fit(data_df)
cleaned = cleaner.transform(data_df)

In [0]:
# Show label and resulting features
cleaned.select(['label', 'features']).show()

In [0]:
from pyspark.ml.classification import NaiveBayes
# Break data down into a training set and a testing set
training, testing = cleaned.randomSplit([0.75, 0.25])

# Create a Naive Bayes model and fit training data
nb = NaiveBayes()
predictor = nb.fit(training)

In [0]:
# Tranform the model with the testing data
test_results = predictor.transform(testing)
test_results.show(10)

In [0]:
# Use the Class Evaluator for a cleaner description
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting reviews was: %f" % acc)